# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.

# Option 1: Jupyter Notebook magic commands

#     %load_ext dotenv
#     %dotenv


# Option 2: Plain Python code

# Load environment variable using dotenv
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Retrieve the PRICE_DATA environment variable
PRICE_DATA = os.getenv('PRICE_DATA')
PRICE_DATA


In [2]:
# Turn the query planning option on to prevent message
import dask
dask.config.set({'dataframe.query-planning': True})
    
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Retrieve the PRICE_DATA environment variable
PRICE_DATA = os.getenv('PRICE_DATA')
assert os.path.isdir(PRICE_DATA), f"'{PRICE_DATA=}' is not a valid directory"

# Get all *.parquet files and directories recursively
parquet_paths = glob(os.path.join(PRICE_DATA, "**", "*.parquet"), recursive=True)

# Filter to keep only files (exclude directories)
parquet_files = [path for path in parquet_paths if os.path.isfile(path)]
assert len(parquet_files) == 11207, f"Expected 11207 files, but found {len(parquet_files)}"

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.

import pandas as pd
import numpy as np

# Read all parquet files into a single Dask DataFrame
ddf = dd.read_parquet(parquet_files).set_index('ticker')

# Provides Dask with a template of the expected output structure, 
# so it knows the columns and data types without computing the 
# entire operation immediately.
# Not strictly necessary, but it's a nice-to-have.
column_types = {
    'Date': 'datetime64[ns, UTC]',
    'Adj Close': float,
    'Close': float,
    'High': float,
    'Low': float,
    'Open': float,
    'Volume': np.int64,
    'sector': 'string[pyarrow]',
    'subsector': 'string[pyarrow]',
    'year': 'int32',
    'Close_lag': float,
    'Adj_Close_lag': float,
    'hi_lo_range': float,
    'returns': float
}
meta_df = pd.DataFrame({col: pd.Series(dtype=dt) for col, dt in column_types.items()})

# Option 1: Add features using chain of apply(), lambda, and assign()
dd_feat = (
    ddf.groupby('ticker', group_keys=False)
    .apply(
        lambda x: x.sort_values('Date').assign(
            # Add lags for 'Close' and 'Adj_Close'
            Close_lag = x['Close'].shift(1),
            Adj_Close_lag = x['Adj Close'].shift(1),

            # Calculate the daily high-low range
            hi_lo_range = x['High'] - x['Low']
        ).assign(
            # Calculate returns based on Adjusted Close
            returns = lambda x: x['Adj Close'] / x['Adj_Close_lag'] - 1
        )
        , meta = meta_df
    )
)

# Option 2: Add features with apply() and externally defined function.
# (See my Student Notes at the bottom of this notebook for details.)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [17]:
# Write your code below.

# Convert the Dask DataFrame to a Pandas DataFrame (takes 2m36s)
pd_feat = dd_feat.compute()

# Calculate the 10-day rolling average return using the Pandas dataframe (takes 1s)
pd_feat['avg_return_10d'] = pd_feat.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(drop=True)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

### Was it necessary to convert to Pandas to calculate the moving average return?

No, it wasn't strictly necessary to convert to Pandas to calculate the moving average return. Dask has support for rolling operations, so we could have calculated the moving average return directly within Dask without converting.

### Would it have been better to do it in Dask? Why?

No, in this particular case it would not, because the data is small enough to fit in memory and the computation is fast [1].

In addition, using Dask for rolling window operations is not as convenient as doing it in Pandas. With Dask, you should ensure that the partition sizes you choose are large enough to avoid boundary issues, but keep in mind that larger partitions can begin to slow down your computations. The data should also be index-aligned to ensure that it’s sorted in the correct order. Dask uses the index to determine which rows are adjacent to one another, so ensuring proper sort order is critical for the correct execution of any calculations on the data. [2]

References:
- [1] Dask. (n.d.). *Dask DataFrame*. Retrieved October 27, 2024, from [https://docs.dask.org/en/stable/dataframe.html#when-not-to-use-dask-dataframes](https://docs.dask.org/en/stable/dataframe.html#when-not-to-use-dask-dataframes)
- [2] Daniel, J. C. (2019). *Data science with Python and Dask* (p. 161). Manning Publications.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ x ] Created a branch with the correct naming convention.
- [ x ] Ensured that the repository is public.
- [ x ] Reviewed the PR description guidelines and adhered to them.
- [ x ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

In [ ]:
# To verify, display the first results
print(pd_feat.head(20))

In [ ]:
# Prevent ValueError: cannot reindex on an axis with duplicate labels
pd_feat = pd_feat.reset_index()

In [ ]:
print(pd_feat[[]'ticker', 'Date', 'returns', 'avg_return_10d']].head(2000))

## Student Notes

*Option 2: Add features with apply() and externally defined function*

This option only works when the function is defined outside the Jupyter notebook, otherwise Dask throws an error: 
```
Function ... may not be deterministically hashed by cloudpickle
```
Here are the steps to use this approach:
1. Define the function in its own source file, outside the notebook. For example, in `${SRC_DIR}/feature_engineering.py`:
    ```python
    # For each ticker, add lags, returns, and high-low range
    def add_features(df):
        # Sort by date if not already sorted
        #df = df.sort_index()
        
        # Add lags for 'Close' and 'Adj_Close'
        df['Close_lag'] = df['Close'].shift(1)
        df['Adj_Close_lag'] = df['Adj Close'].shift(1)
        
        # Calculate returns based on Adjusted Close
        df['returns'] = (df['Adj Close'] / df['Adj_Close_lag']) - 1
        
        # Calculate the daily high-low range
        df['hi_lo_range'] = df['High'] - df['Low']
        
        return df
    ```

2. In the notebook, import the externally defined function and apply it to each group of the Dask dataframe:
    ```python
    import sys
    sys.path.append(os.getenv('SRC_DIR'))

    from feature_engineering import add_features

    dd_feat = ddf.groupby('ticker', group_keys=False).apply(add_features, meta=ddf)
    ```